In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [3]:
# Nombres de los archivos
archivos = ['RIGHT.csv', 'LEFT.csv', 'UP.csv', 'DOWN.csv', 'PRESS.csv']
# Cargar y combinar los archivos
data_df = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

In [5]:
# Normalización de los datos
scaler = StandardScaler()
data_df[['horizontal', 'vertical']] = scaler.fit_transform(data_df[['horizontal', 'vertical']])

In [6]:
# Preparar los datos de entrada y salida
X = data_df[['horizontal', 'vertical']].values
y = to_categorical(data_df['label'].values)  # One-hot encoding para las etiquetas

In [8]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Redimensionar los datos para la CNN 1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [11]:
# Construir el modelo
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(2, 1)))  # kernel_size reducido a 1
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [12]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/10


16641/16641 [==============================] - 253s 15ms/step - loss: 3.7315 - accuracy: 0.0561 - val_loss: 3.6558 - val_accuracy: 0.0652
Epoch 2/10
16641/16641 [==============================] - 243s 15ms/step - loss: 3.6316 - accuracy: 0.0710 - val_loss: 3.6049 - val_accuracy: 0.0740
Epoch 3/10
16641/16641 [==============================] - 258s 16ms/step - loss: 3.5879 - accuracy: 0.0752 - val_loss: 3.5739 - val_accuracy: 0.0767
Epoch 4/10
16641/16641 [==============================] - 238s 14ms/step - loss: 3.5599 - accuracy: 0.0789 - val_loss: 3.5501 - val_accuracy: 0.0799
Epoch 5/10
16641/16641 [==============================] - 248s 15ms/step - loss: 3.5403 - accuracy: 0.0804 - val_loss: 3.5296 - val_accuracy: 0.0850
Epoch 6/10
16641/16641 [==============================] - 267s 16ms/step - loss: 3.5250 - accuracy: 0.0831 - val_loss: 3.5251 - val_accuracy: 0.0831
Epoch 7/10
16641/16641 [==============================] - 270s 16ms/step - loss: 3.5133 - accuracy: 0.08

In [14]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

1301/1301 [==============================] - 12s 9ms/step - loss: 3.4902 - accuracy: 0.0898
Loss: 3.4901907444000244, Accuracy: 0.08980553597211838


In [15]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [16]:
# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

1301/1301 [==============================] - 9s 6ms/step


In [17]:
# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

[[366   0  32 ...  34  90  10]
 [ 68   9  35 ...   0  81  15]
 [ 92   0  34 ...   2  79   6]
 ...
 [ 30   0   2 ... 121  48  13]
 [108   0   6 ...  35 223  28]
 [ 41   4   0 ...   5  59 113]]


In [22]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Construir el modelo LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(y.shape[1], activation='softmax'))

# Compilar
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar
lstm_model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluar
loss_lstm, accuracy_lstm = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Test Loss: {loss_lstm}, Test Accuracy: {accuracy_lstm}')

Epoch 1/100
5201/5201 [==============================] - 86s 14ms/step - loss: 3.7663 - accuracy: 0.0517
Epoch 2/100
5201/5201 [==============================] - 79s 15ms/step - loss: 3.6005 - accuracy: 0.0748
Epoch 3/100
5201/5201 [==============================] - 82s 16ms/step - loss: 3.5248 - accuracy: 0.0846
Epoch 4/100
5201/5201 [==============================] - 86s 17ms/step - loss: 3.4811 - accuracy: 0.0891
Epoch 5/100
5201/5201 [==============================] - 82s 16ms/step - loss: 3.4512 - accuracy: 0.0939
Epoch 6/100
5201/5201 [==============================] - 79s 15ms/step - loss: 3.4304 - accuracy: 0.0955
Epoch 7/100
5201/5201 [==============================] - 79s 15ms/step - loss: 3.4139 - accuracy: 0.0988
Epoch 8/100
5201/5201 [==============================] - 75s 14ms/step - loss: 3.4024 - accuracy: 0.0993
Epoch 9/100
5201/5201 [==============================] - 79s 15ms/step - loss: 3.3918 - accuracy: 0.1012
Epoch 10/100
5201/5201 [==============================]

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(model.history.history['loss_lstm'])

In [25]:
# Realizar predicciones
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

1301/1301 [==============================] - 14s 8ms/step
[[284  25  23 ...  22  35  10]
 [ 69  57  14 ...   1  21  12]
 [ 49  10  77 ...   0  20  25]
 ...
 [ 30   3   5 ... 120  42  13]
 [ 62   8  28 ...  31 195  28]
 [ 14   5   7 ...   6  24 122]]
